<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20RAG_Metrics%26Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index==0.9.14.post3 deeplake==3.8.12 openai==1.3.8 cohere==4.37

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
os.environ["ACTIVELOOP_TOKEN"] = "<YOUR_ACTIVELOOP_TOKEN>"

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

# build service context
llm = OpenAI(model="gpt-4", temperature=0.0)
service_context = ServiceContext.from_defaults(llm=llm)

# FaithfulnessEvaluator

In [ ]:
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

vector_store = DeepLakeVectorStore(dataset_path="hub://genai360/LlamaIndex_paulgraham_essay", overwrite=False)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

Deep Lake Dataset in hub://genai360/LlamaIndex_paulgraham_essay already exists, loading from the storage


In [ ]:
from llama_index.evaluation import FaithfulnessEvaluator

# define evaluator
evaluator = FaithfulnessEvaluator(service_context=service_context)

In [ ]:
# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What does Paul Graham do?"
)

eval_result = evaluator.evaluate_response(response=response)

In [ ]:
print( "> response:", response )

print( "> evaluator result:", eval_result.passing )

> response: Paul Graham is involved in organizing a summer program called the Summer Founders Program, where undergraduates start their own startups. He also owns a building in Cambridge that serves as the program's headquarters. Additionally, Paul Graham is a founder of Y Combinator, a startup accelerator, and he and his team provide seed funding and support to selected startup founders.
> evaluator result: True


# RAGAS

In [ ]:
!pip install html2text==2020.1.16 ragas==0.0.22

In [ ]:
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True).load_data( ["https://en.wikipedia.org/wiki/New_York_City"] )

In [ ]:
from llama_index import VectorStoreIndex, ServiceContext

vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

query_engine = vector_index.as_query_engine()

In [ ]:
response_vector = query_engine.query("How did New York City get its name?")

print(response_vector)

New York City got its name in honor of the Duke of York, who later became King James II of England. The Duke's elder brother, King Charles II, appointed him as the proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control.


In [ ]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,000",  # incorrect answer
    "Queens",  # incorrect answer
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

eval_answers = [[a] for a in eval_answers]

In [ ]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [ ]:
from ragas.llama_index import evaluate

result = evaluate(query_engine, metrics, eval_questions, eval_answers)

evaluating with [faithfulness]


100%|██████████| 1/1 [00:19<00:00, 19.86s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:03<00:00,  3.33s/it]


evaluating with [context_precision]


100%|██████████| 1/1 [00:02<00:00,  2.86s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:07<00:00,  7.10s/it]


evaluating with [harmfulness]


100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


In [ ]:
# final scores
print(result)

{'faithfulness': 0.8000, 'answer_relevancy': 0.7634, 'context_precision': 0.6000, 'context_recall': 0.9000, 'harmfulness': 0.0000}


In [ ]:
result.to_pandas()

,question,answer,contexts,ground_truths,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,What is the population of New York City as of ...,The population of New York City as of 2020 is ...,"[4. ^ _**a**_ _**b**_ [""QuickFacts: New York c...","[8,804,000]",0.0,0.00000,1.0,1.0,0
1,Which borough of New York City has the highest...,Brooklyn is the borough of New York City that ...,[[A map showing five boroughs in different\nco...,[Queens],1.0,1.00000,1.0,0.5,0
2,What is the economic significance of New York ...,New York City has significant economic signifi...,[New York City has been ranked first among cit...,[New York City's economic significance is vast...,1.0,1.00000,1.0,1.0,0
3,How did New York City get its name?,New York City got its name in honor of the Duk...,[[34]\n\n## Etymology\n\nSee also: [Nicknames ...,[New York City got its name when it came under...,1.0,0.96372,0.0,1.0,0
4,What is the significance of the Statue of Libe...,The Statue of Liberty in New York City is a gl...,[[](//upload.wikimedia.org/wikipedia/commons/t...,[The Statue of Liberty in New York City holds ...,1.0,0.85347,0.0,1.0,0


# Custom RAG pipeline

In [ ]:
!wget 'https://raw.githubusercontent.com/idontcalculate/data-repo/main/venus_transmission.txt'

--2023-12-18 21:29:05--  https://raw.githubusercontent.com/idontcalculate/data-repo/main/venus_transmission.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19241 (19K) [text/plain]
Saving to: ‘venus_transmission.txt.2’

venus_transmission. 100%[===================>]  18.79K  --.-KB/s    in 0.001s  

2023-12-18 21:29:05 (27.9 MB/s) - ‘venus_transmission.txt.2’ saved [19241/19241]



In [ ]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["/content/venus_transmission.txt"])

docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

Loaded 1 docs


In [ ]:
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(docs)
vector_index = VectorStoreIndex(nodes)

In [ ]:
query_engine = vector_index.as_query_engine()

response_vector = query_engine.query("What was The first beings to inhabit the planet?")

In [ ]:
response_vector.response

'The first beings to inhabit the planet were a dinoid and reptoid race from two different systems outside our solar system.'

In [ ]:
# First retrieved node
response_vector.source_nodes[0].get_text()

"They had heard of this beautiful new planet. At this time, Earth had two moons to harmonize the weather conditions and control the tides of the large bodies of water.\nThe first beings to inhabit the planet were a dinoid and reptoid race from two different systems outside our solar system. They were intelligent and walked on two legs like humans and were war-like considering themselves to be superior to all other life forms. In the past, the four races of humans had conflicts with them before they outgrew such behavior. They arrived on Earth to rob it of its minerals and valuable gems. Soon they had created a terrible war. They were joined by re-\n1\nenforcements from their home planets. One set up its base on one of the Earth's moons, the other on Earth. It was a terrible war with advanced nuclear and laser weapons like you see in your science fiction movies. It lasted very long. Most of the life forms lay in singed waste and the one moon was destroyed. No longer interested in Earth,

In [ ]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"Due to the radiation, the survivors of the dinoids and reptoids mutated into the Dinosaurs and giant reptilians you know of in your history. The humans that were trapped there mutated into what you call Neanderthals.\nThe Earth remained a devastated ruin, covered by a huge dark nuclear cloud and what vegetation was left was being devoured by the giant beings, also humans and animals by some. It was this way for hundreds of years before a giant comet crashed into one of the oceans and created another huge cloud. This created such darkness that the radiating heat of the Sun could not interact with Earth's gravitational field and an ice age was created. This destroyed the mutated life forms and gave the four races the chance to cleanse and heal the Earth with technology and their energy.\nOnce again, they brought various forms of life to the Earth, creating again a paradise, except for extreme weather conditions and extreme tidal activities.\nDuring this time they realized that their pla

In [ ]:
from llama_index.llms import OpenAI
from llama_index.evaluation import generate_question_context_pairs

# Define an LLM
llm = OpenAI(model="gpt-3.5-turbo")

qa_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=2
)

queries = list(qa_dataset.queries.values())
print( queries )

100%|██████████| 13/13 [00:27<00:00,  2.13s/it]

['Explain the role of different alien races in the history of our solar system according to the information provided. How did these races contribute to the transformation process and why was Earth considered a special planet?', 'Describe the advanced abilities and technology possessed by the Masters and beings mentioned in the context. How did their understanding of creation and their eternal nature shape their perspective on life and death?', 'Explain the concept of creativity as understood by the advanced races mentioned in the context. How did they use their creative energy and what were the responsibilities associated with it?', 'Describe the initial state of Earth before it became a planet and settled into an orbit around the Sun. How did the four races contribute to the development of life on Earth?', 'How did the arrival of the dinoid and reptoid races on Earth lead to a devastating war? Discuss the reasons behind their conflict with the four races of humans and the impact it ha

In [ ]:
retriever = vector_index.as_retriever(similarity_top_k=2)

In [ ]:
from llama_index.evaluation import RetrieverEvaluator

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [ ]:
# Evaluate
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [ ]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [ ]:
import pandas as pd

display_results("OpenAI Embedding Retriever", eval_results)

,Retriever Name,Hit Rate,MRR
0,OpenAI Embedding Retriever,0.884615,0.788462


In [ ]:
from llama_index import ServiceContext

# gpt-3.5-turbo
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context_gpt35 = ServiceContext.from_defaults(llm=gpt35)

# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

In [ ]:
vector_index = VectorStoreIndex(nodes, service_context = service_context_gpt35)
query_engine = vector_index.as_query_engine()

In [ ]:
eval_query = queries[10]

response_vector = query_engine.query(eval_query)

In [ ]:
print( "> eval_query: ", eval_query )
print( "> response_vector:", response_vector )

> eval_query:  How did the colonies respond to the declaration of war by the dark forces, and what measures did they take to protect their knowledge and technology?
> response_vector: The colonies did not fight back against the dark forces when they declared war. Instead, they sent most of their people into hiding in order to rebuild the colonies later. They also destroyed everything to ensure that their knowledge and technology would not fall into the hands of the dark forces. Additionally, Lemuria and Atlantis were destroyed by their inhabitants to prevent the misuse of their knowledge and technology by the dark forces.


In [ ]:
from llama_index.evaluation import RelevancyEvaluator
from llama_index.evaluation import FaithfulnessEvaluator

relevancy_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)
faithfulness_gpt4 = FaithfulnessEvaluator(service_context=service_context_gpt4)

True

In [ ]:
# Compute faithfulness evaluation

eval_result = faithfulness_gpt4.evaluate_response(response=response_vector)
# check passing parameter in eval_result if it passed the evaluation.
print( eval_result.passing )

# Relevancy evaluation
eval_result = relevancy_gpt4.evaluate_response(
    query=eval_query, response=response_vector
)
# You can check passing parameter in eval_result if it passed the evaluation.
print( eval_result.passing )

True
True


In [ ]:
#Batch Evaluator:
#BatchEvalRunner to compute multiple evaluations in batch wise manner.

from llama_index.evaluation import BatchEvalRunner

# Let's pick top 10 queries to do evaluation
batch_eval_queries = queries[:10]

# Initiate BatchEvalRunner to compute FaithFulness and Relevancy Evaluation.
runner = BatchEvalRunner(
    {"faithfulness": faithfulness_gpt4, "relevancy": relevancy_gpt4},
    workers=8,
)

# Compute evaluation
eval_results = await runner.aevaluate_queries(
    query_engine, queries=batch_eval_queries
)
# Let's get faithfulness score
faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
# get relevancy score
relevancy_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['relevancy'])

print( "> faithfulness_score", faithfulness_score )
print( "> relevancy_score", relevancy_score )

> faithfulness_score 1.0
> relevancy_score 1.0
